In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# load stitchnet
# restitching
# evaluate accuracy vs # of samples used to stitch

In [3]:
from stitchnet.stitchonnx.utils import load_cats_and_dogs_dset, load_dl

from tqdm import tqdm
import torch
import numpy as np
import random

dataset_train = load_cats_and_dogs_dset("train")
dataset_val = load_cats_and_dogs_dset("test")

/opt/anaconda3/envs/stitchnet/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from skl2onnx.helpers.onnx_helper import load_onnx_model
from stitchnet.stitchonnx.utils import Net
from glob import glob
from pathlib import Path

netsFiles = sorted(glob('_models/fragments/net*'))
fragmentsMap = {}
for i,netsFile in enumerate(netsFiles):
    fragmentFiles = sorted(glob(str(Path(netsFile)/'fragment*.onnx')))
    onnxFragments = []
    js = []
    for j,fragmentFile in enumerate(fragmentFiles):
        onnxFragment = load_onnx_model(fragmentFile)
        onnxFragments.append(onnxFragment)
        js.append(j)
    net1 = Net(onnxFragments, i)
    for j in js:
        fragmentsMap[(i,j)] = net1[j]

In [5]:
import pandas as pd
EXPNAME = 'result_paper_32'
data = pd.read_csv(f'_results/{EXPNAME}.txt', names=['score','acc','macs','params','path','fragments'])

In [6]:
from ast import literal_eval as make_tuple
from stitchnet.stitchonnx.utils import load_onnx_model
from stitchnet.stitchonnx.utils import stitch_fragments
from stitchnet.stitchonnx.utils import accuracy_score_net
import os

STITCH_BATCH_SIZES = [
    8,32, 64, 128, 256
]
RESULT_NAME = f'stitch_samples_{EXPNAME}'

os.makedirs(f'_results/{RESULT_NAME}', exist_ok=True)
for netId,fragments in enumerate(data['fragments']):
    with open(f'_results/{RESULT_NAME}/{netId:03}.txt', 'w') as file:
        for batch_size in STITCH_BATCH_SIZES:
            # random.seed(51)
            # np.random.seed(24)
            # torch.manual_seed(77)

            dl_score = load_dl(dataset_train, batch_size)
            data_score,t = next(iter(dl_score))
            data_score = data_score.numpy()

            fs = make_tuple(fragments)
            onnx_fragments = []
            for f in fs:
                netId,fId = f
                onnx_fragments.append(fragmentsMap[(netId,fId)])

            # stitching fragments
            fragmentC = onnx_fragments[0]
            for k in range(1,len(onnx_fragments)):
                onnx_model = stitch_fragments(fragmentC, onnx_fragments[k], data_score)
                net = Net([onnx_model])
                fragmentC = net[0]

            acc = accuracy_score_net(net, dataset_val, bs=256)
            print(batch_size,acc)
            file.write(f'{batch_size},{acc}\n')
            file.flush()

/opt/anaconda3/envs/stitchnet/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:54: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn(


epoch 0 loss 8.399702393778678e-10
epoch 1 loss 5.181281625356364e-10
epoch 2 loss 3.360187689745962e-10
epoch 3 loss 2.4102893008707305e-10
epoch 4 loss 1.6567061942700421e-10
epoch 5 loss 1.1249265320506329e-10
epoch 6 loss 8.451944829122127e-11
epoch 7 loss 6.499766797030766e-11
epoch 8 loss 5.228026455585181e-11
epoch 9 loss 4.518009499519212e-11
epoch 0 loss 2.055193922737649e-08
epoch 1 loss 8.1039704703595e-09
epoch 0 loss 1.903628379715463e-08
epoch 1 loss 1.2375302013914244e-08
epoch 2 loss 7.335292725629564e-09
epoch 0 loss 6.426770079954315e-09
epoch 1 loss 5.233966769524347e-09
epoch 2 loss 3.6590461889872558e-09
epoch 3 loss 3.14576928871484e-09
epoch 0 loss 0.0
epoch 1 loss 0.0
epoch 2 loss 0.0
epoch 3 loss 0.0
epoch 4 loss 0.0
epoch 5 loss 0.0
epoch 6 loss 0.0
epoch 7 loss 0.0
epoch 8 loss 0.0
epoch 9 loss 0.0
epoch 0 loss 0.0
epoch 1 loss 0.0
epoch 2 loss 0.0
epoch 3 loss 0.0
epoch 4 loss 0.0
epoch 5 loss 0.0
epoch 6 loss 0.0
epoch 7 loss 0.0
epoch 8 loss 0.0
epoch 9 lo

100%|██████████| 20/20 [01:18<00:00,  3.93s/it]


8 0.7715127701375245
epoch 0 loss 2.386355826250806e-09
epoch 1 loss 5.030680082231636e-10
epoch 2 loss 2.07322602920116e-10
epoch 3 loss 1.6407502090705797e-10
epoch 4 loss 1.5036785777808593e-10
epoch 5 loss 1.4377837497018117e-10
epoch 6 loss 1.4055240799910813e-10
epoch 7 loss 1.379705439974409e-10
epoch 8 loss 1.364696869456328e-10
epoch 9 loss 1.3460797116485724e-10
epoch 0 loss 1.5843284606661973e-08
epoch 1 loss 9.320712596216458e-09
epoch 2 loss 5.835069695013131e-09
epoch 3 loss 3.92575466940319e-09
epoch 4 loss 2.998279830855673e-09
epoch 5 loss 2.4072285624409624e-09
epoch 6 loss 2.0038911647720823e-09
epoch 7 loss 1.8020459654403216e-09
epoch 8 loss 1.6577556947159528e-09
epoch 9 loss 1.5891502709413834e-09
epoch 0 loss 1.226440924885817e-08
epoch 1 loss 8.64296313151359e-09
epoch 2 loss 6.793486903857148e-09
epoch 3 loss 5.781863245904642e-09
epoch 4 loss 5.176099412332965e-09
epoch 5 loss 4.7220084515761276e-09
epoch 6 loss 4.429285277802113e-09
epoch 7 loss 4.2628406839

100%|██████████| 20/20 [01:30<00:00,  4.52s/it]


...

100%|██████████| 20/20 [01:08<00:00,  3.43s/it]


8 0.48860510805500984
epoch 0 loss 1.323516018926677e-09
epoch 1 loss 2.74835433789288e-10
epoch 2 loss 1.436319540670394e-10
epoch 3 loss 1.2002818796792657e-10
epoch 4 loss 1.1156842649861764e-10
epoch 5 loss 1.0649436370088951e-10
epoch 6 loss 1.0352187602802915e-10
epoch 7 loss 1.0150008192567527e-10
epoch 8 loss 1.0007614185602427e-10
epoch 9 loss 9.907764990998735e-11
epoch 0 loss 1.8897307692368595e-08
epoch 1 loss 1.187650118158645e-08
epoch 2 loss 8.621891797486855e-09
epoch 3 loss 5.7589602252018046e-09
epoch 4 loss 4.0728997873211805e-09
epoch 5 loss 3.068763315781939e-09
epoch 6 loss 2.3132547028462634e-09
epoch 7 loss 1.875551422256934e-09
epoch 8 loss 1.6334950538824014e-09
epoch 9 loss 1.5030513480783237e-09
epoch 0 loss 1.5441742443565532e-08
epoch 1 loss 9.717304252510247e-09
epoch 2 loss 7.285082160141153e-09
epoch 3 loss 5.6990067313558105e-09
epoch 4 loss 4.763661559889055e-09
epoch 5 loss 4.164914356854983e-09
epoch 6 loss 3.7469626086936766e-09
epoch 7 loss 3.4926

100%|██████████| 20/20 [01:05<00:00,  3.27s/it]


32 0.6447937131630649
epoch 0 loss 4.142694701836521e-09
epoch 1 loss 5.222255464523231e-10
epoch 2 loss 3.697379954406185e-10
epoch 3 loss 3.4139975467886256e-10
epoch 4 loss 3.2527174491440653e-10
epoch 5 loss 3.1739526842767315e-10
epoch 6 loss 3.136683328865129e-10
epoch 7 loss 3.115148389205243e-10
epoch 8 loss 3.1037889770898353e-10
epoch 9 loss 3.090270115705126e-10
epoch 0 loss 1.3300775140056463e-08
epoch 1 loss 7.023248339244125e-09
epoch 2 loss 4.037842344507883e-09
epoch 3 loss 3.2461573941698307e-09
epoch 4 loss 2.8681048699727184e-09
epoch 5 loss 2.7381423361413864e-09
epoch 6 loss 2.6960449914647878e-09
epoch 7 loss 2.6733135310952797e-09
epoch 8 loss 2.6609761337193738e-09
epoch 9 loss 2.6523821613743043e-09
epoch 0 loss 1.5628864840038778e-08
epoch 1 loss 1.113654315379645e-08
epoch 2 loss 9.526250494554754e-09
epoch 3 loss 8.840020429014922e-09
epoch 4 loss 8.524546769984539e-09
epoch 5 loss 8.39993926880983e-09
epoch 6 loss 8.323058292536893e-09
epoch 7 loss 8.289295

 45%|████▌     | 9/20 [00:34<00:42,  3.82s/it]

KeyboardInterrupt



In [ ]:
# onnx_fragments

In [4]:
# with open('_results/result_paper.txt', 'r') as f:
#     for line in f:
#         print(line)

In [28]:
# onnx_fragments[0].graph.input